In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [4]:
class BottleNeck(layers.Layer):
    def __init__(self, growth_rate, drop_rate):
        super().__init__()
        self.listLayers = [
            layers.BatchNormalization(), 
            layers.ReLU(), 
            layers.Conv2D(4 * growth_rate, kernel_size=1, strides=1, padding='same'), 
            
            layers.BatchNormalization(), 
            layers.ReLU(), 
            layers.Conv2D(growth_rate, kernel_size=3, strides=1, padding='same'), 
            
            layers.Dropout(drop_rate),
        ]
        return 
    
    def call(self, X):
        Y = X
        for layer in self.listLayers.layers:
            Y = layer(Y)
        Y = layers.concatenate([X, Y], axis=-1)
        return Y


In [5]:
class DenseBlock(layers.Layer):
    def __init__(self, num_layers, growth_rate, drop_rate=0.5):
        super().__init__()
        self.listLayers = [
            BottleNeck(growth_rate, drop_rate)
            for _ in range(num_layers)
        ]
        return 
    
    def call(self, X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X

    
blk = DenseBlock(2, 10)
X = tf.random.uniform((4, 8, 8,3))
Y = blk(X)
print(Y.shape)


(4, 8, 8, 23)


In [6]:
class TransitionLayer(layers.Layer):
    def __init__(self, out_channels):
        super().__init__()
        self.listLayers = [
            layers.BatchNormalization(), 
            layers.ReLU(), 
            layers.Conv2D(out_channels, kernel_size=1, strides=1, padding='same'), 
            layers.MaxPool2D(pool_size=2, strides=2, padding='same'),
        ]
        return 
    
    def call(self, X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X

    
blk = TransitionLayer(10)
blk(Y).shape#TensorShape([4, 4, 4, 10])


TensorShape([4, 4, 4, 10])

In [8]:
class DenseNet(keras.Model):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, drop_rate):
        super().__init__()
        self.listLayers = []
        self.listLayers.extend([
            layers.Conv2D(num_init_features, kernel_size=7, strides=2, padding='same'), 
            layers.BatchNormalization(), 
            layers.ReLU(), 
            layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
        ])
        
        num_channels = num_init_features
        for i in range(len(block_layers)):
            self.listLayers.append(DenseBlock(block_layers[i], growth_rate, drop_rate))
            if i < len(block_layers) - 1:
                num_channels = (num_channels + growth_rate * block_layers[i]) * compression_rate
                self.listLayers.append(TransitionLayer(out_channels=int(num_channels)))
                
        self.listLayers.extend([
            layers.GlobalAveragePooling2D(), 
            layers.Dense(10, activation='softmax'),
        ])
        return 
    
    def call(self, X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X
    
net = DenseNet(
    num_init_features=64, growth_rate=32, 
    block_layers=[4,4,4,4], 
    compression_rate=0.5, drop_rate=0.5
)

In [9]:
input_shape = (1, 96, 96, 1)
X = tf.random.uniform(input_shape)
for blk in net.layers:
    X = blk(X)
    print(blk.name, '\t', X.shape)

net.build(input_shape=input_shape)
print(net.summary())

conv2d_5 	 (1, 48, 48, 64)
batch_normalization_5 	 (1, 48, 48, 64)
re_lu_5 	 (1, 48, 48, 64)
max_pooling2d_1 	 (1, 24, 24, 64)
dense_block_1 	 (1, 24, 24, 192)
transition_layer_1 	 (1, 12, 12, 96)
dense_block_2 	 (1, 12, 12, 224)
transition_layer_2 	 (1, 6, 6, 112)
dense_block_3 	 (1, 6, 6, 240)
transition_layer_3 	 (1, 3, 3, 120)
dense_block_4 	 (1, 3, 3, 248)
global_average_pooling2d 	 (1, 248)
dense 	 (1, 10)
Model: "dense_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            multiple                  3200      
_________________________________________________________________
batch_normalization_5 (Batch multiple                  256       
_________________________________________________________________
re_lu_5 (ReLU)               multiple                  0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple

In [14]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 16
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (16, 224, 224, 1) y_batch shape: (16,)


In [15]:
net.compile(
    optimizer=keras.optimizers.Adam(1e-7), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

epochs = 1
num_iter = dataLoader.num_train // batch_size
for ep in range(epochs):
    for n in range(num_iter):
        X_batch, y_batch = dataLoader.get_batch_train(batch_size)
        net.fit(X_batch, y_batch)
        if n % 20 == 0:
            net.save_weights('densenet_weights.h5')
            

Train on 16 samples
16/16 [==============================] - 6s 387ms/sample - loss: 3.1896 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 3.2114 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3766 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6761 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3215 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.9803 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7697 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7352 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4932 - accuracy: 0.1875
T

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4745 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5122 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7811 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.9176 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4080 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6287 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7394 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6048 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7504 - accuracy: 0.1250
Tr

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7582 - accuracy: 0.2500
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7371 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7052 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8528 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 11ms/sample - loss: 3.0051 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6771 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3308 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6410 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7792 - accuracy: 0.0000e+0

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.1937 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.1433 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6646 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8629 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7616 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8242 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3173 - accuracy: 0.2500
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5482 - accuracy: 0.2500
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4664 - accuracy: 0.1250
Tr

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5809 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6085 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2948 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8246 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6737 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3213 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5928 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7328 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7369 - accuracy: 0.1875
Tr

...

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4304 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5157 - accuracy: 0.2500
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7510 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8759 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5305 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6883 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3356 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6919 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.8676 - accuracy: 0.125

16/16 [==============================] - 0s 10ms/sample - loss: 2.6379 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.1598 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2597 - accuracy: 0.3125
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7693 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7873 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2786 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5849 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5239 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2927 - accuracy: 0.1875
Train on 16 samples
16/16 

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7394 - accuracy: 0.2500
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5319 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4906 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5401 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6876 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7320 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6213 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3688 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5911 - accuracy: 0.0000e+00
Tr

16/16 [==============================] - 0s 10ms/sample - loss: 2.3094 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3230 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2947 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4819 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5777 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6249 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6101 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4187 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4716 - accuracy: 0.0625
Train on 16 samples
16

Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.6406 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.4462 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.2963 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.1064 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5227 - accuracy: 0.0625
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3650 - accuracy: 0.0000e+00
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.3866 - accuracy: 0.1875
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.7044 - accuracy: 0.1250
Train on 16 samples
16/16 [==============================] - 0s 10ms/sample - loss: 2.5262 - accuracy: 0.1250
Tr

In [16]:
net.load_weights('densenet_weights.h5')
X_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(X_test, y_test, verbose=2)

2000/2000 - 6s - loss: 2.3299 - accuracy: 0.0995


[2.3299410247802737, 0.0995]